In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
common = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas5.csv', encoding='utf-8')
common.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante
0,1112363488,8.617918e+06,12,1,34,15677,0,0,0,0,1,151,34,1,356027
1,1112299213,1.101489e+07,9,2,24,9371,0,0,0,0,0,182,127,1,396236
2,1112425871,6.232296e+06,16,2,25,10775,0,0,0,0,1,94,838,1,125047
3,1112208765,1.005179e+07,11,2,32,15848,0,0,0,0,0,5,139,1,158791
4,1112377689,4.856018e+06,10,1,26,20003,0,0,0,0,0,182,391,1,29737


In [ ]:
common.drop_duplicates()

In [2]:
ad20180603 = pd.read_csv('../data/adri_data_20180603/top100k.csv', encoding='utf-8')
ad20180603.rename({'sepostulo' : 'sp1', 'id': 'id1'}, axis = 1, inplace=True)
ad20180603.head()

,id1,sp1
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [3]:
ad20180604 = pd.read_csv('../data/adri_data/top100k.csv', encoding='utf-8')
ad20180604.rename({'sepostulo' : 'sp2', 'id': 'id2'}, axis=1, inplace=True)
ad20180604.head()

,id2,sp2
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [4]:
ad = pd.concat([ad20180603, ad20180604], axis = 1).reset_index(drop=True)
ad.drop('id2', inplace=True, axis=1)
ad.rename({'id1' : 'id'}, axis=1, inplace=True)

In [5]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')

In [6]:
top100 = pd.merge(top100, ad, on='id')
top100.head()

,id,idaviso,idpostulante,sp1,sp2
0,0,739260,6M9ZQR,0,0
1,1,739260,6v1xdL,0,0
2,2,739260,ezRKm9,0,0
3,3,758580,1Q35ej,0,0
4,4,758580,EAN4J6,0,0


In [7]:
id_postulantes = pd.read_csv('../data/adri_data/entrenamiento_id_postulantes.csv', encoding='utf-8')
id_postulantes.shape
id_postulantes.head()

,idpostulante,idpostulante2
0,pzeVW63,365446
1,PmVbBWY,209736
2,zvPMxvJ,409297
3,YYwJKq,250105
4,3NkMAVq,37024


In [8]:
top100_full = pd.merge(top100, id_postulantes, how='left', on='idpostulante')
top100_full.head()

,id,idaviso,idpostulante,sp1,sp2,idpostulante2
0,0,739260,6M9ZQR,0,0,63651.0
1,1,739260,6v1xdL,0,0,76877.0
2,2,739260,ezRKm9,0,0,323397.0
3,3,758580,1Q35ej,0,0,NaN
4,4,758580,EAN4J6,0,0,127342.0


In [9]:
top100_full.drop('idpostulante', inplace=True, axis=1)
# top100_full.rename({'idpostulante':'idpostulante_old'}, inplace=True, axis=1)
top100_full.rename({'idpostulante2':'idpostulante'}, inplace=True, axis=1)
top100_full.shape

(100000, 5)

In [10]:
top100_full.head()

,id,idaviso,sp1,sp2,idpostulante
0,0,739260,0,0,63651.0
1,1,739260,0,0,76877.0
2,2,739260,0,0,323397.0
3,3,758580,0,0,NaN
4,4,758580,0,0,127342.0


In [11]:
df = pd.read_csv('../data/adri_data/entrenamiento_postulaciones_vistas4.csv', encoding='utf-8')
df.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante
0,1112480068,3.797336e+06,9,2,37,11852,0,0,0,0,3,29,351,0,268089
1,1112252447,8.747997e+06,11,2,33,11848,0,0,0,0,3,182,160,1,64325
2,1112370570,5.677360e+06,5,-1,-1,6406,0,0,0,0,0,130,544,0,329658
3,1112297862,8.325712e+06,12,2,61,18872,0,0,0,0,0,182,25,0,278279
4,1112417632,5.190836e+06,12,2,28,320,0,0,0,0,0,40,22,0,151183


In [12]:
df = pd.merge(df, top100_full, how='left', on=['idpostulante', 'idaviso'])

In [13]:
df.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante,id,sp1,sp2
0,1112480068,3.797336e+06,9,2,37,11852,0,0,0,0,3,29,351,0,268089,NaN,NaN,NaN
1,1112252447,8.747997e+06,11,2,33,11848,0,0,0,0,3,182,160,1,64325,NaN,NaN,NaN
2,1112370570,5.677360e+06,5,-1,-1,6406,0,0,0,0,0,130,544,0,329658,NaN,NaN,NaN
3,1112297862,8.325712e+06,12,2,61,18872,0,0,0,0,0,182,25,0,278279,NaN,NaN,NaN
4,1112417632,5.190836e+06,12,2,28,320,0,0,0,0,0,40,22,0,151183,NaN,NaN,NaN


In [14]:
df.shape

(15849097, 18)

In [15]:
df.drop('id', axis=1, inplace=True)

In [16]:
df.fillna(0, inplace=True)

In [17]:
df

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo,idpostulante,sp1,sp2
0,1112480068,3.797336e+06,9,2,37,11852,0,0,0,0,3,29,351,0,268089,0.0,0.0
1,1112252447,8.747997e+06,11,2,33,11848,0,0,0,0,3,182,160,1,64325,0.0,0.0
2,1112370570,5.677360e+06,5,-1,-1,6406,0,0,0,0,0,130,544,0,329658,0.0,0.0
3,1112297862,8.325712e+06,12,2,61,18872,0,0,0,0,0,182,25,0,278279,0.0,0.0
4,1112417632,5.190836e+06,12,2,28,320,0,0,0,0,0,40,22,0,151183,0.0,0.0
5,1112455550,4.163061e+06,12,1,29,15727,0,0,0,0,2,12,4188,1,274262,0.0,0.0
6,1112430528,5.816944e+06,12,2,23,1690,0,0,0,0,0,182,15,1,214243,0.0,0.0
7,1112461610,4.164174e+06,6,2,32,14112,0,0,0,0,1,136,22,0,358286,0.0,0.0
8,1112478169,3.829434e+06,16,1,27,8820,0,0,0,0,0,29,49,0,286696,0.0,0.0
9,1112419049,5.964277e+06,6,2,34,9045,0,0,0,0,0,2,1093,1,397655,0.0,0.0


In [18]:
# X will have the numeric data, Y will have the target class
dfix = df[['idaviso','timestamp', 'valor_educacion', 'sexo', 'edad', 'idtitulo', 'idzona', 'idciudad','idmapacalle', 
           'idtipo_de_trabajo', 'idnivel_laboral', 'idnombre_area', 'iddenominacion_empresa', 'idpostulante', 'sp1', 'sp2']]
X = np.array(dfix)
y = np.array(df['se_postulo']) 

In [19]:
# Split into train and test, use only 10% as test (15 flowers)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=7506)
X_test

array([[1.11241799e+09, 5.11303088e+06, 1.00000000e+01, ...,
        2.66063000e+05, 0.00000000e+00, 0.00000000e+00],
       [1.11240536e+09, 4.82727892e+06, 6.00000000e+00, ...,
        3.18833000e+05, 0.00000000e+00, 0.00000000e+00],
       [1.11233479e+09, 8.48426317e+06, 6.00000000e+00, ...,
        3.18201000e+05, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.11196337e+09, 9.37610317e+06, 6.00000000e+00, ...,
        1.54480000e+05, 0.00000000e+00, 0.00000000e+00],
       [1.11239412e+09, 6.06777050e+06, 1.00000000e+01, ...,
        5.16110000e+04, 0.00000000e+00, 0.00000000e+00],
       [1.11239836e+09, 5.74231517e+06, 1.20000000e+01, ...,
        1.26444000e+05, 0.00000000e+00, 0.00000000e+00]])

In [20]:
y_test

array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [21]:
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

In [22]:
# FIT (this does nothing in normal KNN)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [23]:
# predict the response
pred = knn.predict(X_test)
pred.shape

(1584910,)

In [24]:
# evaluate accuracy
print (accuracy_score(y_test, pred)) # 3 = 0.8889779230366393
# evaluate accuracy
print (accuracy_score(y_test, pred)) # 5 = 0.8934198156362191

0.8934198156362191
0.8934198156362191


In [25]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,0
1,0
2,1
3,0
4,0


# Predigo con todos los datos

In [137]:
top100 = pd.read_csv("../test_final_100k.csv", encoding='utf-8')

In [138]:
id_postulantes = pd.read_csv('../data/adri_data/entrenamiento_id_postulantes.csv', encoding='utf-8')
id_postulantes.shape
id_postulantes.head()

,idpostulante,idpostulante2
0,pzeVW63,365446
1,PmVbBWY,209736
2,zvPMxvJ,409297
3,YYwJKq,250105
4,3NkMAVq,37024


In [139]:
top100_full = pd.merge(top100, id_postulantes, how='left', on='idpostulante')
top100_full.head()

,id,idaviso,idpostulante,idpostulante2
0,0,739260,6M9ZQR,63651.0
1,1,739260,6v1xdL,76877.0
2,2,739260,ezRKm9,323397.0
3,3,758580,1Q35ej,NaN
4,4,758580,EAN4J6,127342.0


In [140]:
# top100_full.drop('idpostulante', inplace=True, axis=1)
top100_full.rename({'idpostulante':'idpostulante_old'}, inplace=True, axis=1)
top100_full.rename({'idpostulante2':'idpostulante'}, inplace=True, axis=1)
top100_full.shape

(100000, 4)

In [141]:
top100_full.head()

,id,idaviso,idpostulante_old,idpostulante
0,0,739260,6M9ZQR,63651.0
1,1,739260,6v1xdL,76877.0
2,2,739260,ezRKm9,323397.0
3,3,758580,1Q35ej,NaN
4,4,758580,EAN4J6,127342.0


In [142]:
top100_full[top100_full['idpostulante'].isnull()].shape

(15334, 4)

In [143]:
dfix2 = pd.merge(top100_full, dfix, how='inner', on=['idaviso','idpostulante'])
dfix2.shape

(97985, 16)

In [158]:
dfix2.head()

,id,idaviso,idpostulante_old,idpostulante,sepostulo,contador
0,594,17903700,Yj5q3xX,252853,0,1
1,594,17903700,Yj5q3xX,252853,0,1
2,594,17903700,Yj5q3xX,252853,0,1
3,594,17903700,Yj5q3xX,252853,0,1
4,594,17903700,Yj5q3xX,252853,0,1


In [144]:
dfix3 = dfix2[['idaviso','timestamp', 'valor_educacion', 'sexo', 'edad', 'idtitulo', 'idzona', 'idciudad','idmapacalle', 
           'idtipo_de_trabajo', 'idnivel_laboral', 'idnombre_area', 'iddenominacion_empresa', 'idpostulante']]

In [ ]:
dfix3[dfix3.duplicated(['idaviso', 'idpostulante'])]

In [145]:
dfix3.head()

,idaviso,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,idpostulante
0,17903700,4.600113e+06,12,1,37,9407,0,0,0,0,0,122,1130,252853
1,17903700,4.580593e+06,12,1,37,9407,0,0,0,0,0,122,1130,252853
2,17903700,4.600116e+06,12,1,37,9407,0,0,0,0,0,122,1130,252853
3,17903700,4.189065e+06,12,1,37,9407,0,0,0,0,0,122,1130,252853
4,17903700,4.600057e+06,12,1,37,9407,0,0,0,0,0,122,1130,252853


In [146]:
X_full_data = np.array(dfix3)
X_full_data

array([[17903700, 4600112.658955, 12, ..., 122, 1130, 252853.0],
       [17903700, 4580592.834955, 12, ..., 122, 1130, 252853.0],
       [17903700, 4600116.173955, 12, ..., 122, 1130, 252853.0],
       ...,
       [1112477683, 3343630.395955, 12, ..., 2, 1427, 313128.0],
       [1112485441, 3538380.353955, 18, ..., 14, 686, 313527.0],
       [1112498129, 3029073.648955, 10, ..., 2, 1206, 24730.0]],
      dtype=object)

In [147]:
pred_full_data = knn.predict(X_full_data)

In [148]:
dfix2['sepostulo'] = pred_full_data

In [149]:
dfix2.drop(['valor_educacion', 'timestamp', 'sexo','edad','idtitulo','idzona','idciudad','idmapacalle',
            'idtipo_de_trabajo','idnivel_laboral','idnombre_area','iddenominacion_empresa'], inplace=True, axis=1)
dfix2.head()

,id,idaviso,idpostulante_old,idpostulante,sepostulo
0,594,17903700,Yj5q3xX,252853,0
1,594,17903700,Yj5q3xX,252853,0
2,594,17903700,Yj5q3xX,252853,0
3,594,17903700,Yj5q3xX,252853,0
4,594,17903700,Yj5q3xX,252853,0


In [150]:
dfix2[dfix2['id'] == 594]

,id,idaviso,idpostulante_old,idpostulante,sepostulo
0,594,17903700,Yj5q3xX,252853,0
1,594,17903700,Yj5q3xX,252853,0
2,594,17903700,Yj5q3xX,252853,0
3,594,17903700,Yj5q3xX,252853,0
4,594,17903700,Yj5q3xX,252853,0
5,594,17903700,Yj5q3xX,252853,0
6,594,17903700,Yj5q3xX,252853,0
7,594,17903700,Yj5q3xX,252853,0


In [151]:
dfix2['contador'] = 1

In [152]:
grouped = dfix2.groupby(['id', 'idaviso', 'idpostulante', 'idpostulante_old'])['sepostulo', 'contador'].sum().reset_index()
grouped.head()

,id,idaviso,idpostulante,idpostulante_old,sepostulo,contador
0,594,17903700,252853.0,Yj5q3xX,0,8
1,595,17903700,222828.0,RzrDJlZ,0,1
2,609,1000139175,170021.0,LNKrDLr,0,3
3,610,1000139175,323242.0,exv4B8,3,3
4,637,1000468151,325851.0,jkMko3r,0,2


In [153]:
grouped['prob'] = grouped['sepostulo'] / grouped['contador']
grouped

In [154]:
grouped.head()

,id,idaviso,idpostulante,idpostulante_old,sepostulo,contador,prob
0,594,17903700,252853.0,Yj5q3xX,0,8,0.000000
1,595,17903700,222828.0,RzrDJlZ,0,1,0.000000
2,609,1000139175,170021.0,LNKrDLr,0,3,0.000000
3,610,1000139175,323242.0,exv4B8,3,3,1.000000
4,637,1000468151,325851.0,jkMko3r,0,2,0.000000
5,640,1000502125,149673.0,GNoLbRN,0,2,0.000000
6,655,1000686071,136761.0,EzZ9pl6,0,2,0.000000
7,702,1001135716,330476.0,kP0MvPQ,1,15,0.066667
8,704,1001135716,335727.0,kPjaNo0,1,4,0.250000
9,705,1001135716,125409.0,DrzxMGX,0,2,0.000000


In [155]:
grouped[(grouped['prob']> 0)&(grouped['prob']< 1)]

,id,idaviso,idpostulante,idpostulante_old,sepostulo,contador,prob
7,702,1001135716,330476.0,kP0MvPQ,1,15,0.066667
8,704,1001135716,335727.0,kPjaNo0,1,4,0.250000
17,723,1001284385,84252.0,8MP5p9W,1,3,0.333333
32,854,1002471470,280634.0,aj1v0,1,3,0.333333
35,859,1002471470,220052.0,RzM9VOo,1,3,0.333333
42,872,1002471470,283048.0,ak5dAEE,1,2,0.500000
45,875,1002471470,104237.0,96z3WY8,1,3,0.333333
46,877,1002471470,216198.0,QNr8qG5,1,2,0.500000
54,895,1002490524,213775.0,QNP1kbN,1,2,0.500000
55,896,1002490524,289707.0,akjve0J,1,2,0.500000


In [116]:
grouped['sepostulo2'] = grouped['prob'] >= 0.5

In [117]:
grouped['sepostulo'] = grouped['prob'].apply(lambda x: 1 if x else 0)
grouped['sepostulo']

0        0
1        0
2        0
3        1
4        0
5        0
6        0
7        1
8        1
9        0
10       0
11       0
12       1
13       0
14       1
15       0
16       0
17       1
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       1
28       0
29       0
        ..
38585    1
38586    1
38587    1
38588    0
38589    1
38590    1
38591    1
38592    1
38593    1
38594    1
38595    1
38596    1
38597    1
38598    1
38599    1
38600    1
38601    1
38602    1
38603    1
38604    1
38605    1
38606    1
38607    1
38608    1
38609    0
38610    0
38611    0
38612    0
38613    0
38614    0
Name: sepostulo, Length: 38615, dtype: int64

In [118]:
grouped.drop(['prob', 'sepostulo2', 'contador'], axis=1, inplace=True)
grouped

,id,idaviso,idpostulante,idpostulante_old,sepostulo
0,594,17903700,252853.0,Yj5q3xX,0
1,595,17903700,222828.0,RzrDJlZ,0
2,609,1000139175,170021.0,LNKrDLr,0
3,610,1000139175,323242.0,exv4B8,1
4,637,1000468151,325851.0,jkMko3r,0
5,640,1000502125,149673.0,GNoLbRN,0
6,655,1000686071,136761.0,EzZ9pl6,0
7,702,1001135716,330476.0,kP0MvPQ,1
8,704,1001135716,335727.0,kPjaNo0,1
9,705,1001135716,125409.0,DrzxMGX,0


In [119]:
grouped = dfix2.groupby(['id', 'idaviso', 'idpostulante', 'idpostulante_old'])['sepostulo'].max().reset_index()
grouped.head()

,id,idaviso,idpostulante,idpostulante_old,sepostulo
0,594,17903700,252853.0,Yj5q3xX,0
1,595,17903700,222828.0,RzrDJlZ,0
2,609,1000139175,170021.0,LNKrDLr,0
3,610,1000139175,323242.0,exv4B8,1
4,637,1000468151,325851.0,jkMko3r,0


In [120]:
grouped.shape

(38615, 5)

In [121]:
merged = pd.merge(grouped, df, how='inner', on=['idaviso', 'idpostulante'])
merged.head()

,id,idaviso,idpostulante,idpostulante_old,sepostulo,timestamp,valor_educacion,sexo,edad,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,se_postulo
0,594,17903700,252853,Yj5q3xX,0,4.600113e+06,12,1,37,9407,0,0,0,0,0,122,1130,0
1,594,17903700,252853,Yj5q3xX,0,4.580593e+06,12,1,37,9407,0,0,0,0,0,122,1130,0
2,594,17903700,252853,Yj5q3xX,0,4.600116e+06,12,1,37,9407,0,0,0,0,0,122,1130,0
3,594,17903700,252853,Yj5q3xX,0,4.189065e+06,12,1,37,9407,0,0,0,0,0,122,1130,1
4,594,17903700,252853,Yj5q3xX,0,4.600057e+06,12,1,37,9407,0,0,0,0,0,122,1130,0


In [122]:
# Nuevos predichos
merged[(merged['sepostulo'] == 1) & (merged['se_postulo'] == 0)].shape

(42267, 18)

In [123]:
# Viejos que hay que volver a acomodar
merged[(merged['sepostulo'] == 0) & (merged['se_postulo'] == 1)].shape

(6904, 18)

In [124]:
# Acomodo los viejos como deben estar
merged.loc[(merged['sepostulo'] == 0) & (merged['se_postulo'] == 1), 'sepostulo'] = 1
merged[(merged['sepostulo'] == 0) & (merged['se_postulo'] == 1)]['sepostulo']

Series([], Name: sepostulo, dtype: int64)

In [125]:
merged = merged.groupby(['id', 'idaviso', 'idpostulante'])['sepostulo'].max().reset_index()
merged.head()

,id,idaviso,idpostulante,sepostulo
0,594,17903700,252853.0,1
1,595,17903700,222828.0,0
2,609,1000139175,170021.0,1
3,610,1000139175,323242.0,1
4,637,1000468151,325851.0,1


In [126]:
merged.shape

(38615, 4)

In [127]:
merged2 = pd.merge(top100_full, merged, on=['id', 'idpostulante', 'idaviso'], how='left')
merged2.head()

,id,idaviso,idpostulante_old,idpostulante,sepostulo
0,0,739260,6M9ZQR,63651.0,NaN
1,1,739260,6v1xdL,76877.0,NaN
2,2,739260,ezRKm9,323397.0,NaN
3,3,758580,1Q35ej,NaN,NaN
4,4,758580,EAN4J6,127342.0,NaN


In [128]:
merged2.shape

(100000, 5)

In [129]:
merged2.fillna(0, inplace=True)

In [130]:
merged2[merged2.duplicated('id')].shape

(0, 5)

In [131]:
merged2[merged2['sepostulo'] == 1].shape

(33822, 5)

In [132]:
merged2[merged2['sepostulo'] == 0].shape

(66178, 5)

In [133]:
merged2.drop(['idaviso', 'idpostulante', 'idpostulante_old'], axis=1, inplace=True)

In [134]:
merged2['sepostulo'] = merged2['sepostulo'].astype(int)

In [135]:
merged2.head()

,id,sepostulo
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [136]:
merged2.to_csv('../data/adri_data/top100k.csv', encoding='utf-8', index=False)